In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt




In [ ]:
# Set the path to your dataset
dataset_path = "/path/to/your/dataset"

# Set the path to your metadata file
metadata_path = "/path/to/your/metadata.csv"



In [ ]:
# Load the metadata
metadata_df = pd.read_csv(metadata_path)

# Function to load and process audio files
def load_and_process_audio(file_path, duration=2.5, sampling_rate=22050):
    try:
        # Load audio file
        audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=duration, sr=sampling_rate)
        
        # Extract Mel-frequency cepstral coefficients (MFCCs)
        mfccs = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=13)
        
        return mfccs
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None



In [ ]:
# Extract audio features for each audio file
audio_features = []
labels = []

for index, row in metadata_df.iterrows():
    filename = row['filename']
    emotion_class = row['emotion_class']

    file_path = os.path.join(dataset_path, emotion_class, filename)
    audio_feature = load_and_process_audio(file_path)

    if audio_feature is not None:
        audio_features.append(audio_feature)
        labels.append(emotion_class)

# Convert the lists to NumPy arrays
X = np.array(audio_features)
y = np.array(labels)



In [ ]:
# Encode the emotion labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)



In [ ]:
# Build a simple Convolutional Neural Network (CNN) model
model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2], 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



In [ ]:
# Reshape input data for CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))



In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true_labels, y_pred_labels)
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)

# Plot training history
plt.figure(figsize=(12, 5))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')

plt.show()